In [17]:
import plotly.express as pe
from plotly.figure_factory import create_distplot
import pandas as pd
import datetime as dt
import seaborn as sns
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.dates as mdates
import matplotlib.cbook as cbook
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def hatchSpecies_River(hatchery, species, river):
    #Create df from hatchery file
    esc = pd.read_csv('wdfwHatchery6_18.csv', low_memory=False)
    esc['Date'] = pd.to_datetime(esc['Date'])
    esc = esc[['Date', 'Facility', 'Adult Count', 'Species', 'Event']]
    esc = esc[esc.Facility == hatchery]
    esc = esc[esc.Event == 'Trap Estimate']
    esc = esc[esc.Species == species]
  
    
    #Creat df from rivers file
    df = pd.read_csv(river, sep='\t', header=None)
    df = df.rename(columns ={2:"Date",3:"cfs"})
    df = df.drop(columns=[0,1,4],)
    df['Date'] = df['Date'].astype('datetime64[ns]')
    
    #Specify date range for each species
    df_time = esc
    first = df_time.iloc[0].Date
    last = df_time.iloc[-1].Date
    
    #Merge data frames and trim for species date range
    plt_tst = pd.merge(df, esc, on=['Date'], how='outer')
    rt = (plt_tst['Date'] >= first) & (plt_tst['Date'] <= last)
    plt_tst = plt_tst.loc[rt]

    #Some data formatting
    plt_tst['DOY'] = plt_tst['Date'].dt.strftime('%j')
    plt_tst['Month'] = plt_tst['Date'].dt.strftime('%b')
    plt_tst['Month#'] = plt_tst['Date'].dt.strftime('%m')
    plt_tst['Year'] = plt_tst['Date'].dt.strftime('%Y')
    plt_tst['Adult Count'] = plt_tst['Adult Count'].fillna(0)
    plt_tst['returnBinary'] = plt_tst['Adult Count'].astype(np.int64)
    plt_tst.returnBinary[plt_tst.returnBinary > 0] = 1
    plt_tst['Facility'] = hatchery
    plt_tst['Species'] = species
    return(plt_tst)

In [18]:
river = 'sky.txt'
df = hatchSpecies_River('WALLACE R HATCHERY', 'Chinook', river)

In [19]:
df.head()

,Date,cfs,Facility,Adult Count,Species,Event,DOY,Month,Month#,Year,returnBinary
5117,2011-06-29,11300.0,WALLACE R HATCHERY,54.0,Chinook,Trap Estimate,180,Jun,06,2011,1
5118,2011-06-30,12600.0,WALLACE R HATCHERY,0.0,Chinook,NaN,181,Jun,06,2011,0
5119,2011-07-01,10100.0,WALLACE R HATCHERY,0.0,Chinook,NaN,182,Jul,07,2011,0
5120,2011-07-02,9520.0,WALLACE R HATCHERY,0.0,Chinook,NaN,183,Jul,07,2011,0
5121,2011-07-03,10700.0,WALLACE R HATCHERY,0.0,Chinook,NaN,184,Jul,07,2011,0


In [32]:
def dataviz(indata):
    pfig = make_subplots(specs=[[{"secondary_y": True}]])

    pfig.add_trace(go.Scatter(y=indata.cfs, x=indata.Date,
                              mode='lines',
                              name='cfs'))
    pfig.add_trace(go.Scatter(y=indata['Adult Count'], x=indata.Date,
                              mode='lines',
                              name='returns'),
                              secondary_y=True,
                  )
    return(pfig)



In [33]:
dataviz(hatchSpecies_River('REITER PONDS', 'Steelhead', river))